In [1]:
!pip install grakel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.5 MB/s eta 0:00:00


In [5]:
# NetLSD
!pip install netlsd
import netlsd

  Preparing metadata (setup.py) ... done
  Created wheel for netlsd: filename=NetLSD-1.0.2-py3-none-any.whl size=6749 sha256=27b7f72ec04caf15a3868578cfbf1e15f3883754a66ae872d3d02d994130703e
  Stored in directory: /root/.cache/pip/wheels/1e/20/fc/625057cbf6fb45226db3d699df0f5666162a81e81c8eca9eca
Successfully built netlsd


In [2]:
import json
import itertools as it
from collections import defaultdict
from time import time

import numpy as np
import networkx as nx


In [3]:

def grakel2nx(g):
    return nx.from_edgelist(g.get_edges())

class Kernel:
    def __init__(self,normalize=True):
        self.normalize=normalize

    '''
        A function that maps a networkx graph to a numpy vector
    '''
    def embed(self,nxg):
        pass

    '''
        Returns the dotproduct if normalize==False and the cosine similarity otherwise
    '''
    def sim(self,v1,v2):
        dot = np.dot(v1,v2)
        if not self.normalize:
            return dot
        return dot / (np.linalg.norm(v1)*np.linalg.norm(v2))

    def fit_transform(self,pack):
        representations = None
        if self.embed_dataset is not None:
            representations = self.embed_dataset(map(grakel2nx,pack))
        else:
            representations = [
                self.embed(grakel2nx(grakelg))
                for grakelg in pack
            ]
        return np.array([
            [
                self.sim(v1,v2)
                for v2 in representations
            ]
            for v1 in representations
        ])


class NetLSD(Kernel):
    def __init__(self, normalize=True):
        super().__init__(normalize)
        import netlsd
        self.embed = netlsd.heat
        self.embed_dataset = None

class Gin(Kernel):
    def __init__(self, normalize=True):
        super().__init__(normalize)
        from scipy import spatial
        import torch
        import dgl
        from GgmMetrics.evaluation import gin_evaluation
        device =  torch.device('cpu')
        model = gin_evaluation.load_feature_extractor(device)
        ev = gin_evaluation.MMDEvaluation(model=model, kernel='rbf', sigma='range', multiplier='mean')
        self.embed = lambda x: ev._GINMetric__get_activations_single_dataset([dgl.DGLGraph(x)])[0]
        self.embed_dataset = lambda x: ev._GINMetric__get_activations_single_dataset(list(map(dgl.DGLGraph,x)))



In [4]:
# from generate_graphs import ig2edges,edges2grakel
from grakel.kernels import (RandomWalk,
                            GraphletSampling,
                            PyramidMatch,
                            NeighborhoodHash,
                            ShortestPath,
                            WeisfeilerLehman,
                            Propagation,
                            OddSth,
                            WeisfeilerLehmanOptimalAssignment,
                            NeighborhoodSubgraphPairwiseDistance)
# from other_kernels import NetLSD

selected_kernels = (
     RandomWalk, # ERRRs,
     GraphletSampling,
     NetLSD,
     PyramidMatch,
     NeighborhoodHash,
     ShortestPath,
     WeisfeilerLehman,
     Propagation,
     OddSth,
     WeisfeilerLehmanOptimalAssignment,
     NeighborhoodSubgraphPairwiseDistance,
     # SvmTheta, # ERRR
    )



In [6]:
from grakel import Graph
from grakel.utils import graph_from_networkx

slices = defaultdict(set)
names = []

gk_graphs = []
for idx, g in enumerate(nx.graph_atlas_g()):
    if not idx:
      continue
    if not len(g.edges()):
      continue
    v_n = len(g.nodes())
    if not v_n:
      continue
    conn = nx.is_connected(g)
    gn = str(g).split("'")[1]
    names.append( gn )
    slices['full'].add(idx)
    slices[f'size_{v_n}'].add(idx)
    if conn:
        slices['connected'].add(idx)
        slices[f'connected_size_{v_n}'].add(idx)
    else:
        slices['non_connected'].add(idx)
        slices[f'non_connected_size_{v_n}'].add(idx)

    nx.set_node_attributes(g, dict([(i,'A') for i in range(v_n)]), 'label')
    nx.set_edge_attributes(g, {tuple(ee): {'elabel':'B'} for ee in g.edges()})
    gg = list(graph_from_networkx([g,], node_labels_tag='label', edge_labels_tag='elabel', as_Graph=True))[0] # , as_Graph=True)
    gk_graphs.append( gg )

kernel_params = defaultdict(dict)
kernel_params[GraphletSampling] = {
    'sampling': {'n_samples': 500}
}
kernel_params[RandomWalk] = {
    'lamda': 0.1, # not 'lambda' (typo in grakel?)
    'p': 5
}

kernel_vals = dict()
for k in selected_kernels:
    print(k.__name__)
    vals = k(normalize=True,**kernel_params[k]).fit_transform(gk_graphs)
    print((vals == 1.).sum(), len(gk_graphs))
    kernel_vals[k.__name__] = vals

kernel_vals['GraphletSampling_without_sampling'] = GraphletSampling(normalize=True).fit_transform(gk_graphs)

pairs = defaultdict(list)
for k in kernel_vals:
  for i, r in enumerate(kernel_vals[k].tolist()):
    for j, c in enumerate(r):
      if i>j and c == 1.:
        pairs[k].append((names[i],names[j]))
# with open('vals.json', 'w') as ofh:
#   print(json.dumps(kernel_vals), file=ofh)
with open('pairs.json', 'w') as ofh:
  print(json.dumps(pairs), file=ofh)

RandomWalk
1321 1245
GraphletSampling
1245 1245
NetLSD


/usr/local/lib/python3.10/dist-packages/grakel/graph.py:313: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


901 1245
PyramidMatch
1249 1245
NeighborhoodHash
23479 1245
ShortestPath
42535 1245
WeisfeilerLehman
1359 1245
Propagation
1103977 1245
OddSth
1257 1245
WeisfeilerLehmanOptimalAssignment
1297 1245
NeighborhoodSubgraphPairwiseDistance


/usr/local/lib/python3.10/dist-packages/grakel/kernels/neighborhood_subgraph_pairwise_distance.py:314: RuntimeWarning: invalid value encountered in divide
  S += np.nan_to_num(K / np.sqrt(np.outer(K_diag, K_diag)))


12 1245


/usr/local/lib/python3.10/dist-packages/grakel/kernels/graphlet_sampling.py:324: RuntimeWarning: invalid value encountered in divide
  return np.divide(km, np.sqrt(np.outer(self._X_diag, self._X_diag)))


In [7]:
slices = defaultdict(set)
nslices = defaultdict(set)
names = []

gk_graphs = []
for idx, g in enumerate(nx.graph_atlas_g()):
    if not idx:
      continue
    if not len(g.edges()):
      continue
    v_n = len(g.nodes())
    if not v_n:
      continue
    conn = nx.is_connected(g)
    gn = str(g).split("'")[1]
    names.append( gn )
    slices['full'].add(idx)
    slices[f'size_{v_n}'].add(idx)
    if conn:
        slices['connected'].add(idx)
        slices[f'connected_size_{v_n}'].add(idx)
    else:
        slices['non_connected'].add(idx)
        slices[f'non_connected_size_{v_n}'].add(idx)
    nslices['full'].add(gn)
    nslices[f'size_{v_n}'].add(gn)
    if conn:
        nslices['connected'].add(gn)
        nslices[f'connected_size_{v_n}'].add(gn)
    else:
        nslices['non_connected'].add(gn)
        nslices[f'non_connected_size_{v_n}'].add(gn)

pairs = json.loads(open('pairs.json').read())

def cn(n):
    return n*(n-1)//2

def cc(p, s):
    return len([1 for a,b in p if a in s and b in s])

def ccc(p, s):
    return len([1 for a,b in p if (a,b) in s])

def cnss(p='size_'):
    f = set()
    for s in nslices:
        if s.startswith(p):
            for i, a in enumerate(names):
                for j, b in enumerate(names):
                    if i>j and a in nslices[s] and b in nslices[s]:
                        f.add((a,b))
    return f

q = cnss()
qq = cnss('connected_size_')


In [8]:
skip = 0
if not skip:
    for k in pairs:
        print(
            "\t".join(map(str,
            [
                k,
                len(slices['full']),
                cn(len(slices['full'])),
                cc(pairs[k], nslices['full']),
                f"{cc(pairs[k], nslices['full'])/cn(len(slices['full'])):.04%}",
                '--',
                len(slices['connected']),
                cn(len(slices['connected'])),
                cc(pairs[k], nslices['connected']),
                f"{cc(pairs[k], nslices['connected'])/cn(len(slices['connected'])):.04%}",
                '--',
                len(slices['non_connected']),
                cn(len(slices['non_connected'])),
                cc(pairs[k], nslices['non_connected']),
                f"{cc(pairs[k], nslices['non_connected'])/cn(len(slices['non_connected'])):.04%}",
                '--',
                len(q),
                ccc(pairs[k], q),
                f"{ccc(pairs[k], q)/len(q):.04%}",
                '--',
                len(qq),
                ccc(pairs[k], qq),
                f"{ccc(pairs[k], qq)/len(qq):.04%}",
            ]))
            )


RandomWalk	1245	774390	38	0.0049%	--	995	494515	26	0.0053%	--	250	31125	4	0.0129%	--	555914	19	0.0034%	--	369820	16	0.0043%
NetLSD	1245	774390	184	0.0238%	--	995	494515	10	0.0020%	--	250	31125	85	0.2731%	--	555914	27	0.0049%	--	369820	10	0.0027%
PyramidMatch	1245	774390	2	0.0003%	--	995	494515	1	0.0002%	--	250	31125	1	0.0032%	--	555914	2	0.0004%	--	369820	1	0.0003%
NeighborhoodHash	1245	774390	11117	1.4356%	--	995	494515	7570	1.5308%	--	250	31125	739	2.3743%	--	555914	11117	1.9998%	--	369820	7570	2.0469%
ShortestPath	1245	774390	20645	2.6660%	--	995	494515	15794	3.1938%	--	250	31125	1392	4.4723%	--	555914	16934	3.0462%	--	369820	14808	4.0041%
WeisfeilerLehman	1245	774390	57	0.0074%	--	995	494515	37	0.0075%	--	250	31125	7	0.0225%	--	555914	26	0.0047%	--	369820	20	0.0054%
Propagation	1245	774390	551366	71.2000%	--	995	494515	494515	100.0000%	--	250	31125	9091	29.2080%	--	555914	409438	73.6513%	--	369820	369820	100.0000%
OddSth	1245	774390	6	0.0008%	--	995	494515	0	0.0000%	--	250	31125	2	

In [10]:
print(len(list(enumerate(nx.graph_atlas_g()))))

1253


In [11]:
pairs['GraphletSampling_without_sampling']

[['G74', 'G36'],
 ['G76', 'G38'],
 ['G81', 'G31'],
 ['G87', 'G40'],
 ['G88', 'G41'],
 ['G89', 'G42'],
 ['G90', 'G43'],
 ['G91', 'G44'],
 ['G92', 'G72'],
 ['G94', 'G35'],
 ['G97', 'G36'],
 ['G97', 'G74'],
 ['G103', 'G99'],
 ['G105', 'G31'],
 ['G105', 'G81'],
 ['G107', 'G45'],
 ['G108', 'G46'],
 ['G109', 'G47'],
 ['G110', 'G48'],
 ['G130', 'G36'],
 ['G130', 'G74'],
 ['G130', 'G97'],
 ['G131', 'G49'],
 ['G132', 'G50'],
 ['G155', 'G51'],
 ['G174', 'G43'],
 ['G174', 'G90'],
 ['G175', 'G44'],
 ['G175', 'G91'],
 ['G176', 'G52'],
 ['G204', 'G50'],
 ['G204', 'G132'],
 ['G208', 'G52'],
 ['G208', 'G176'],
 ['G220', 'G77'],
 ['G221', 'G79'],
 ['G222', 'G31'],
 ['G222', 'G81'],
 ['G222', 'G105'],
 ['G229', 'G72'],
 ['G229', 'G92'],
 ['G230', 'G35'],
 ['G230', 'G94'],
 ['G231', 'G36'],
 ['G231', 'G74'],
 ['G231', 'G97'],
 ['G231', 'G130'],
 ['G232', 'G37'],
 ['G233', 'G38'],
 ['G233', 'G76'],
 ['G234', 'G77'],
 ['G234', 'G220'],
 ['G236', 'G79'],
 ['G236', 'G221'],
 ['G238', 'G80'],
 ['G240', 'G31']